In [ ]:
import sys 
sys.path.append("../")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.nn import BCELoss

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from pathlib import *
import os
import tqdm
import argparse
import json

from models.tcn import MS_TCN, SS_TCN
from dataset.VolvoDataset import VolvoDatasetPart1, VolvoDatasetPart2
from utils.ContinuityCrossEntropyLoss import ContinuityCrossEntropyLoss
from utils.StatsComputer import StatsComputer

In [ ]:
train_dataset = VolvoDatasetPart1(data_path=train_data_path, variants_path=variants_path)
processor = train_dataset.get_processor()
label_encoder = processor.risk_encoder

train_dataset, validation_dataset = train_dataset.split_train_validation()

test_dataset = VolvoDatasetPart1(data_path=test_data_path, variants_path=variants_path, test=True)
test_dataset.set_processor(processor) 

In [ ]:
# load state dict from checkpoint file

state_dict = torch.load("../../TCN_best.pth")
weights_to_keep = [x for x in state_dict.keys() if "mlp" not in x]

new_state_dict = {}
for key in weights_to_keep:
    new_state_dict[key] = state_dict[key]

new_state_dict.keys()

model_2 = SS_TCN(num_input_channels=train_dataset.get_n_features(), 
                            num_classes=1, 
                            is_phase_1=False
                            )

In [ ]:
model_2.load_state_dict(new_state_dict, strict=False)

In [ ]:
tensor = torch.tensor([0.1, 0.2, 0, 0.4, 0.5])
tensor_1 = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5])

def tensor_to_vector(tensor):
    tensor = tensor * 10
    tensor = torch.round(tensor)
    tensor

    # for each number, create a tensor with 10 elements, with (1 - number) being 0 and number being 1
    list = []
    for n in tensor:
        list.append(torch.cat([torch.zeros(int(10 - n)), torch.ones(int(n))]))

    return torch.stack(list)

tensor = tensor_to_vector(tensor)
tensor_1 = tensor_to_vector(tensor_1)

# count total accuracy

correct = torch.eq(tensor, tensor_1).sum().item()
total = tensor.numel()
correct / total
    

In [ ]:
tensor, tensor_1

In [ ]:
args = argparse.Namespace(
    seed=13_04_2000,
    test_only=False,
    load_model="",
    batch_size=256,
    num_epochs=30,
    learning_rate=0.0005,
    lr_scheduler_gamma=0.8,
    lr_scheduler_step=1,
    patience_epochs=7,
    disable_cuda=False,
    data_path=r"/data1/malto/volvo_ecml_2024",
    train_csv=r"train_gen1.csv",
    test_csv=r"public_X_test.csv",
    variants_csv=r"variants.csv",
    tcnn_weights=r"./"
)

In [ ]:
args = args

np.random.seed(args.seed)
torch.manual_seed(args.seed)

### Get important paths
dataset_path = args.data_path 
variants_path = os.path.join(dataset_path, args.variants_csv)
train_data_path = os.path.join(dataset_path, args.train_csv)
test_data_path = os.path.join(dataset_path, args.test_csv)
weights_path = args.tcnn_weights
os.makedirs(weights_path, exist_ok=True)

### Get dataset and model type
train_data_path = os.path.join(args.data_path, "train_gen1.csv")
test_data_path = os.path.join(args.data_path, "public_X_test.csv")
variants_path = os.path.join(args.data_path, "variants.csv")

In [ ]:
train_dataset = VolvoDatasetPart1(data_path=train_data_path, variants_path=variants_path)
processor = train_dataset.get_processor()
label_encoder = processor.risk_encoder

train_dataset, validation_dataset = train_dataset.split_train_validation()

test_dataset = VolvoDatasetPart1(data_path=test_data_path, variants_path=variants_path, test=True)
test_dataset.set_processor(processor) 

In [ ]:
x, x_static, y = train_dataset[0]

In [ ]:
y

In [ ]:
train_dataset.volvo_df

In [ ]:
"""train_dataset = VolvoDatasetPart2(data_path=train_data_path, variants_path=variants_path)
processor = train_dataset.get_processor()
label_encoder = processor.risk_encoder

train_dataset, validation_dataset = train_dataset.split_train_validation()

test_dataset = VolvoDatasetPart2(data_path=test_data_path, variants_path=variants_path, test=True)
test_dataset.set_processor(processor) """

In [ ]:
n_features = train_dataset.get_n_features()
num_classes = train_dataset.get_n_classes()

#check if preprocess is giving some problems
assert train_dataset.get_n_features() == test_dataset.get_n_features()

model = SS_TCN(     num_input_channels=n_features, 
                    num_classes=num_classes, 
                    is_phase_1=True)

### Get device
device = torch.device(
            "cuda" if (torch.cuda.is_available() and not args.disable_cuda) else "cpu"
        )
model.to(device)
print(f"Working on {device}")

# Load weights if necessary
if args.load_model != "":
    if not(args.load_model.endswith(".pth") or args.load_model.endswith(".pt")):
        raise Exception("Weights file should end with .pt or .pth")
    model_path = os.join.path(weights_path, args.load_model)
    print(f"Loading Model from {model_path}")
    model.load_state_dict(
        torch.load( model_path )
    )

# Create DataLoader instances for train, validation, and test sets
train_loader = DataLoader(train_dataset, 
                                batch_size=args.batch_size, 
                                shuffle=True,
                                num_workers=12) #pin_memory=True #consigliano
val_loader = DataLoader(validation_dataset, 
                                batch_size=args.batch_size, 
                                shuffle=True,
                                num_workers=12)
test_loader =  DataLoader(test_dataset, 
                                batch_size=args.batch_size)

# Define criterion
print('Computing class weights...', end='')
weights = train_dataset.get_weights()
criterion = BCELoss(weight=torch.Tensor(weights).to(device))
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
print('done')
print('Class weights = ', weights)
# criterion = ContinuityCrossEntropyLoss(weights=torch.Tensor([1,1,1]).to(device))

In [ ]:
print("=== Start training ===")
print(f"Batch size: {args.batch_size}")
# Define loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, gamma=args.lr_scheduler_gamma, step_size=args.lr_scheduler_step)
softmax = torch.nn.functional.softmax
waiting_epochs = 0
best_val_loss = float('inf')
best_val_f1 = 0
num_resets = 0
for epoch in range(args.num_epochs):
    ### Run epoch
    print( "="*25, f"EPOCH {epoch}", "="*25)
    print("Learning rate: ", optimizer.param_groups[0]['lr'])
    print("=== TRAIN ===")
    model.train()     
    pbar = tqdm.tqdm(train_loader)
    running_loss = 0
    i = 0
    for timeseries, variants, labels in pbar:
        pbar.set_description(f"Running loss: {running_loss/(i+1e-5) :.4}")
        timeseries, variants, labels = timeseries.to(device), variants.to(device), labels.to(device)
        outputs = model(timeseries, variants)
        outputs = softmax(outputs, dim=-1)
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        i += 1

    epoch_loss = running_loss / (i+1e-5)
    scheduler.step()
    print(f"Epoch [{epoch+1}/{args.num_epochs}], Train Loss: {epoch_loss:.4f}")

    print("=== VAL ===")
    model.eval()
    with torch.no_grad():
        pbar = tqdm.tqdm(val_loader)
        running_loss = 0
        running_acc = 0
        i = 0
        stats = StatsComputer()
        for timeseries, variants, labels in pbar:
            pbar.set_description(f"Running loss: {running_loss/(i+1e-5) :.4}")
            
            timeseries, variants, labels = timeseries.to(device), variants.to(device), labels.to(device)

            outputs = model(timeseries, variants)
            outputs = softmax(outputs, dim=-1)
            loss = criterion(outputs, labels)
            
            # outputs = outputs[-1]
            acc = torch.sum(torch.argmax(labels, dim = 1) == torch.argmax(outputs, dim = 1))
            
            running_acc += acc 

            stats.append(outputs=torch.argmax(outputs, dim=-1).cpu().tolist(), 
                        labels=torch.argmax(labels, dim=-1).cpu().tolist())

            i += 1

        validation_loss = running_loss / i
        validation_accuracy = running_acc / i
        
        validation_f1 = stats.macro_avg_f1()
        print(stats)
    
    print(f"Validation Accuracy: {validation_accuracy:.4f}")
    print(f"Validation Loss: {validation_loss:.4f} vs Best {best_val_loss:.4f}")
    print(f"Validation F1: {validation_f1} vs Best {best_val_f1}")


In [ ]:
from sklearn.metrics import f1_score, precision_recall_fscore_support
def flatten(xss):
    return [x for xs in xss for x in xs]
    
precision, recall, f1, true_sum = precision_recall_fscore_support(flatten(stats.all_labels), flatten(stats.all_outputs)),